In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import math

In [44]:
summarized = pd.read_csv("./results/summarized/1734322688_models_train_test_400.csv", usecols=['id', 'level', 'splitter', 'start_time', 'end_time'])
summarized = summarized[['id','level', 'splitter', 'start_time', 'end_time']]

summarized

times = pd.read_csv("./results/times_01/1734322688_80_SimplestCNNClassifier_8layers_Residual_times.csv")
# times = times[['level', 'splitter', 'start_time', 'end_time']]

# summarized.join(times, lsuffix="_l", rsuffix="_r")
times = pd.merge(summarized, times, on=["level","splitter"], suffixes=["_summarized","_times"])
times

,id_summarized,level,splitter,start_time_summarized,end_time_summarized,Unnamed: 0,experiment_id,id_times,model,batch_size,acc,start_time_times,pred_time,end_time_times
0,0,class,prop_0-1/min_5/RandomSplit_0,1.734323e+09,1.734328e+09,0,1734322688,0,SimplestCNNClassifier_8layers_Residual,15000,0.989240,1.738692e+09,1.738692e+09,1.738693e+09
1,1,class,prop_0-1/min_5/RandomSplit_14,1.734329e+09,1.734334e+09,1,1734322688,1,SimplestCNNClassifier_8layers_Residual,15000,0.989726,1.738693e+09,1.738693e+09,1.738693e+09
2,2,class,prop_0-1/min_5/RandomSplit_56,1.734334e+09,1.734340e+09,2,1734322688,2,SimplestCNNClassifier_8layers_Residual,15000,0.990454,1.738693e+09,1.738693e+09,1.738693e+09
3,3,class,prop_0-1/min_5/RandomSplit_84,1.734340e+09,1.734346e+09,3,1734322688,3,SimplestCNNClassifier_8layers_Residual,15000,0.992072,1.738693e+09,1.738693e+09,1.738693e+09
4,4,class,prop_0-1/min_5/RandomSplit_92,1.734346e+09,1.734352e+09,4,1734322688,4,SimplestCNNClassifier_8layers_Residual,15000,0.991506,1.738693e+09,1.738693e+09,1.738693e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,class,prop_0-05/min_10/StratifiedSplit2_84,1.734766e+09,1.734772e+09,75,1734322688,75,SimplestCNNClassifier_8layers_Residual,15000,0.992065,1.738694e+09,1.738694e+09,1.738694e+09
76,76,class,prop_0-05/min_10/StratifiedSplit2_92,1.734773e+09,1.734779e+09,76,1734322688,76,SimplestCNNClassifier_8layers_Residual,15000,0.990607,1.738695e+09,1.738695e+09,1.738695e+09
77,77,class,prop_0-05/min_10/StratifiedSplit2_101,1.734779e+09,1.734785e+09,77,1734322688,77,SimplestCNNClassifier_8layers_Residual,15000,0.989312,1.738695e+09,1.738695e+09,1.738695e+09
78,78,class,prop_0-05/min_10/StratifiedSplit2_105,1.734785e+09,1.734791e+09,78,1734322688,78,SimplestCNNClassifier_8layers_Residual,15000,0.989150,1.738695e+09,1.738695e+09,1.738695e+09


In [46]:
# times = pd.read_csv("./results/times_01/1734322688_80_SimplestCNNClassifier_8layers_Residual_times.csv")

times[["Prop", "Min", "Splitter_Seed"]] = times["splitter"].str.split("/", expand=True)
times

times[["Splitter","Seed"]] = times["Splitter_Seed"].str.split("_", expand=True)
times

times = times.drop(index = times.loc[(times["Prop"] == "prop_0-05") & (times["Min"] == "min_5")].index)
times

times["Train_Time"] = times["end_time_summarized"] - times["start_time_summarized"]
times["Prediction_Time"] = times["end_time_times"] - times["start_time_times"]

times[['experiment_id', 'level', 'acc', 'Prop', 'Min', 'Splitter', "Train_Time", 'Prediction_Time']]

,experiment_id,level,acc,Prop,Min,Splitter,Train_Time,Prediction_Time
0,1734322688,class,0.989240,prop_0-1,min_5,RandomSplit,5600.479617,23.076327
1,1734322688,class,0.989726,prop_0-1,min_5,RandomSplit,5684.736922,18.173003
2,1734322688,class,0.990454,prop_0-1,min_5,RandomSplit,5666.545494,18.459389
3,1734322688,class,0.992072,prop_0-1,min_5,RandomSplit,5727.036316,19.888611
4,1734322688,class,0.991506,prop_0-1,min_5,RandomSplit,5826.750788,19.372388
...,...,...,...,...,...,...,...,...
75,1734322688,class,0.992065,prop_0-05,min_10,StratifiedSplit2,6116.621783,15.387923
76,1734322688,class,0.990607,prop_0-05,min_10,StratifiedSplit2,6132.624384,17.195623
77,1734322688,class,0.989312,prop_0-05,min_10,StratifiedSplit2,6116.594207,15.282871
78,1734322688,class,0.989150,prop_0-05,min_10,StratifiedSplit2,6105.357798,15.266941


In [47]:
times = times.groupby(by=["level", "Prop"]).agg({
    "Train_Time":["min","mean","max"],
    "Prediction_Time":["min","mean","max"],
}).reset_index()

times

level       Prop   Train_Time                           Prediction_Time  \
                             min         mean          max             min   
0  class  prop_0-05  6090.422589  6132.281750  6210.519334       15.195702   
1  class   prop_0-1  5600.479617  5844.456780  6018.458421       18.173003   
2  class   prop_0-2  5522.091702  5582.913504  5719.413998       31.936493   

                         
        mean        max  
0  15.807418  17.924307  
1  20.459998  25.272042  
2  32.699147  33.704381

In [51]:
times["Train_Time_min"] = times.loc[:,("Train_Time", "min")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Train_Time_max"] = times.loc[:,("Train_Time", "max")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Train_Time_mean"] = times.loc[:,("Train_Time", "mean")].apply(lambda x: str(timedelta(seconds=math.floor(x))))

times["Prediction_Time_min"] = times.loc[:,("Prediction_Time", "min")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Prediction_Time_max"] = times.loc[:,("Prediction_Time", "max")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Prediction_Time_mean"] = times.loc[:,("Prediction_Time", "mean")].apply(lambda x: str(timedelta(seconds=math.floor(x))))

times = times.sort_values(by=[("Prediction_Time", "mean")], ascending=False)[["level","Prop","Train_Time_min","Train_Time_max","Train_Time_mean","Prediction_Time_min","Prediction_Time_max","Prediction_Time_mean"]]

In [52]:
times

,level,Prop,Train_Time_min,Train_Time_max,Train_Time_mean,Prediction_Time_min,Prediction_Time_max,Prediction_Time_mean
,,,,,,,,
2,class,prop_0-2,1:32:02,1:35:19,1:33:02,0:00:31,0:00:33,0:00:32
1,class,prop_0-1,1:33:20,1:40:18,1:37:24,0:00:18,0:00:25,0:00:20
0,class,prop_0-05,1:41:30,1:43:30,1:42:12,0:00:15,0:00:17,0:00:15
